# LSSTCam pointing drift

This notebook calculates the drift of an image due to the pointing error.

Craig Lage - 29-Oct-25

In [ ]:
import numpy as np
from scipy.stats import linregress
import matplotlib.pyplot as plt
import pickle as pkl
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst.daf.butler import Butler
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION
from lsst.obs.lsst import LsstCam
from lsst.summit.utils.efdUtils import calcNextDay
from lsst.geom import SpherePoint,Angle,Extent2I,Box2I,Extent2D,Point2D, Point2I
import lsst.summit.utils.butlerUtils as butlerUtils
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId

In [ ]:
butler = butlerUtils.makeDefaultButler("LSSTCam")
#butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all'])
instrument = 'LSSTCam'
camera = LsstCam.getCamera()
client = makeEfdClient()

In [ ]:
def DeltaAltAz(ra, dec, pressure, hum, temperature, wl, time1, time2, printOut=False):
    # This calculates the change in AltAz during an exposure
    # given the RA/Dec and other variables
    skyLocation = SkyCoord(ra*u.deg, dec*u.deg)
    altAz1 = AltAz(obstime=time1, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    altAz2 = AltAz(obstime=time2, location=SIMONYI_LOCATION, pressure=pressure, 
                 temperature=temperature, relative_humidity=hum, obswl=wl)
    obsAltAz1 = skyLocation.transform_to(altAz1)
    obsAltAz2 = skyLocation.transform_to(altAz2)
    # 1 is at the beginning of the exposure, 2 is at the end
    # el, az are the actual values, prime values reflect the pointing model
    # These are all in degrees
    el1 = obsAltAz1.alt.deg
    az1 = obsAltAz1.az.deg
    el2 = obsAltAz2.alt.deg
    az2 = obsAltAz2.az.deg
    # Change values are the change from the beginning to the end of the exposure, in arcseconds
    azChange = (az2 - az1) * 3600.0
    elChange = (el2 - el1) * 3600.0
    if printOut:
        print(f"AzStart={az1:.6f}, ElStart={el1:.6f}, AzEnd={az2:.6f}, ElEnd={el2:.6f}")
        print(f"AzChange={azChange:.2f}, elChange={elChange:.2f}")
    return [azChange, elChange]

In [ ]:
# Below is the wavelength center point for the LSST filters:
wavelengths = {'u':3671, 'g':4827, 'r':6223, 'i':7546, 'z':8691, 'y':9712}
detector = camera['R22_S11']
bbox = detector.getBBox()

def CalculateDrift(client, expId, deltaT = 0.5, printOut=False):
    # Get the azChange and elChange from the EFD
    dataId = {'exposure':expId, 'instrument':instrument}
    expRecord = getExpRecordFromDataId(butler, dataId)
    (mountErrors, mountData) = calculateMountErrors(expRecord, client)
    azimuthData = mountData.azimuthData
    elevationData = mountData.elevationData
    azValues = np.asarray(azimuthData["actualPosition"])
    azValTimes = np.asarray(azimuthData["actualPositionTimestamp"])
    elValues = np.asarray(elevationData["actualPosition"])
    elValTimes = np.asarray(elevationData["actualPositionTimestamp"])
    startStop = np.asarray([expRecord.timespan.begin.unix_tai + deltaT, expRecord.timespan.end.unix_tai - deltaT])
    azValStartStop = np.interp(startStop, azValTimes, azValues)
    elValStartStop = np.interp(startStop, elValTimes, elValues)
    azChangePoint = (azValStartStop[1] - azValStartStop[0]) * 3600.0
    elChangePoint = (elValStartStop[1] - elValStartStop[0]) * 3600.0
    rawExp = butler.get('raw', detector=94, exposure=expId, instrument=instrument)
    md = rawExp.getMetadata()
    filter = md['FILTBAND']
    wl = wavelengths[filter] * u.angstrom
    pressure = md['PRESSURE'] * u.pascal
    temperature = md['AIRTEMP'] * u.Celsius
    hum = md['HUMIDITY'] / 100.0
    time1 = Time(md['MJD-BEG'], format='mjd', scale='tai') + TimeDelta(deltaT, format='sec')
    time2 = Time(md['MJD-END'], format='mjd', scale='tai') - TimeDelta(deltaT, format='sec')
    raPoint = md['RA']
    decPoint = md['DEC']
    el = md['ELSTART']
    calExp = butler.get('preliminary_visit_image', detector=94, visit=expId, instrument=instrument)
    cWcs = calExp.getWcs()
    if not cWcs:
        return None
    calExpSkyCenter = cWcs.pixelToSky(Point2D(bbox.centerX, bbox.centerY))
    raReal = calExpSkyCenter.getRa().asDegrees()
    decReal = calExpSkyCenter.getDec().asDegrees()
    deltaRa = (raReal - raPoint) * 3600.0
    deltaDec = (decReal - decPoint) * 3600.0
    [azChangeReal, elChangeReal] = DeltaAltAz (raReal, decReal, pressure, hum, temperature, wl, time1, time2, printOut)
    azDrift = azChangeReal - azChangePoint
    azDrift *= np.cos(el * np.pi / 180.0)
    elDrift = elChangeReal - elChangePoint
    totalDrift = np.sqrt(elDrift**2 + azDrift**2)
    if printOut:
        print(f"We think telescope is pointed at (RA, Dec), ({raPoint:.6f}, {decPoint:.6f})")
        print(f"Telescope is actually pointed at (RA, Dec), ({raReal:.6f}, {decReal:.6f})")
        print(f"Pointing error in RA, Dec is ({deltaRa:.1f}, {deltaDec:.1f}) arcseconds")
        print(f"azChangeEFD = {azChangePoint:.2f}, elChangeEFD = {elChangePoint:.2f}")
        print(f"azChangeReal = {azChangeReal:.2f}, elChangeReal = {elChangeReal:.2f}")
        print(f"For {expId}, Azimuth drift = {azDrift:.2f} arcseconds, Elevation drift = {elDrift:.2f} arcseconds, Total drift = {totalDrift:.2f} arcseconds.")
        print(f"Header AzStart={md['AZSTART']:.6f}, AzEnd={md['AZEND']:.6f}, ElStart={md['ELSTART']:.6f}, ElEnd={md['ELEND']:.6f}")
        print(f"Header AzChange={((md['AZEND'] - md['AZSTART'])*3600):.6f}, ElChange={((md['ELEND'] - md['ELSTART'])*3600):.6f}")
    return [azDrift, elDrift]

## Get the guider drift data from RubinTV and calculate the ptg drift

In [ ]:
startDay = 20251110
endDay = 20251116
ptg_az_drifts = []
ptg_el_drifts = []
ptg_total_drifts = []
expIds = []
dayObs = startDay
while dayObs <= endDay:
        exposureList = []
        for record in butler.registry.queryDimensionRecords("exposure", 
                    where=f"exposure.day_obs={dayObs} and instrument='LSSTCam'"):
            exposureList.append([record.id, record])
        exposureList.sort(key=lambda x: x[0])
        print(dayObs, len(exposureList))
        for [id,record] in exposureList:
            if record.observation_type not in ['acq', 'science']:
                continue
            try:
                expId = record.id
                [ptg_az_drift, ptg_el_drift] = CalculateDrift(client, expId)
                if not ptg_az_drift:
                    print(f"{expId} had no WCS!")
                    continue
                expIds.append(expId)
                ptg_az_drifts.append(ptg_az_drift)
                ptg_el_drifts.append(ptg_el_drift)
                ptg_total_drift = np.sqrt(ptg_el_drift**2 + ptg_az_drift**2)
                ptg_total_drifts.append(ptg_total_drift)
                print(f"{expId} succeeded!")
            except Exception as e:
                print(f"Failed with error: {e}")
                print(f"{expId} failed!")
                continue
        print(f" Done with {dayObs}")
        dayObs = calcNextDay(dayObs)     


In [ ]:
filename = f"/home/c/cslage/u/MTMount/mount_plots/pointing_drifts_no_guider_{startDay}_{endDay}.pkl"
with open(filename, 'wb') as f:
    pkl.dump([expIds, ptg_az_drifts, ptg_el_drifts, ptg_total_drifts], f)


## Plot the total drifts

In [ ]:
gt_0p4 = [dr for dr in ptg_total_drifts if dr > 0.4]
pct_gt_0p4 = len(gt_0p4) / len(ptg_total_drifts) * 100    

plt.figure(figsize=(10,5))
plt.suptitle(f"Calculated drift due to pointing model errors {startDay}-{endDay}")
plt.hist(ptg_total_drifts, bins=50, range=(0,1.0))
plt.text(0.6, 150, f"{pct_gt_0p4:.1f} % of visits\n are > 0.4 arcseconds")
plt.xlabel("Total drift/exposure (arcsec)")
plt.savefig(f"/home/c/cslage/u/LSSTCam/guider_data/Pointing_Error_Drifts_Total_{startDay}_{endDay}.png")

In [ ]:
CalculateDrift(client, 2025111900066, printOut=True)